In [1]:
import pickle
import numpy as np
import networkx as nx

N = 1000 # 1000 data points in this training set
k = 10 # Each node in the graph will have edges to its 10 nearest neighbors
S = 50 # need S-nn graph for soft label empirical distribution over S nearest neighbors, S >> k
M = 4 # partition this into 4 parts
num_points = 8 # each example has 8 points
num_features_per_point = 2 # each point has 2 features

folder = '../data/balanced_toy/'

### Create k-NN Graph

In [2]:
# Load the dataset of points
points = pickle.load(open(folder + 'points.pickle', 'rb'))
assert len(points) == N
for point in points:
    # every point is a 8x2 feature
    assert point.shape[0] == num_points
    assert point.shape[1] == num_features_per_point

# Build the k-nearest neighbor graph from these nodes
knn_graph = nx.Graph()
for i, point in enumerate(points):
    knn_graph.add_node(i, reading=point)

# Load the pairwise distances
distances = pickle.load(open(folder + 'distances.pickle', 'rb'))
assert distances.shape == (N, N)

# Compute the k and S nearest neighbors by sorting the distances
S_nearest_neighbors = {}
for i in range(N):
    distances_for_point = list(zip(range(N), distances[i]))
    distances_for_point.sort(key=lambda x: x[1])

    # Add an edge to each of the closest neighbors, skipping the node itself
    for j in range(1, k + 1):
        if not knn_graph.has_edge(i, distances_for_point[j][0]):
            knn_graph.add_edge(i, distances_for_point[j][0], weight=distances_for_point[j][0])

    # Keep track of the exactly S nearest neighbors - need this for soft labels
    S_nearest_neighbors[i] = [i]
    for j in range(1, S):
        S_nearest_neighbors[i].append(distances_for_point[j][0])

nx.write_gpickle(knn_graph, folder + 'knn_graph.gpickle')
pickle.dump(S_nearest_neighbors, open(folder + 'nearest_neighbors.pickle', 'wb'))

### Partition Graph using KL

In [4]:
from networkx.algorithms import community

# import the knn graph
G = nx.read_gpickle(folder + 'knn_graph.gpickle')

# we use a hierarchical partitioning approach to generate m balanced partitions of the graph
# specifically, we choose m to be a power of 2 and repeatedly use Kernighan-Lin to bisect the 
# graph into 2 approximately equal parts
def graph_partition(graph, level):
    if level == 0:
        return [set(graph.nodes())]
    else:
        part1, part2 = community.kernighan_lin_bisection(graph, weight='weight')
        print("done with iteration at level: {}".format(level))
        return graph_partition(graph.subgraph(part1), level - 1) + graph_partition(graph.subgraph(part2), level - 1)

# First, partition the knn graph formed by the training set
num_levels = int(np.log2(M))
assert M == 2 ** num_levels
partition = graph_partition(G, num_levels)

# check the partition was correct
union = []
for part in partition:
    union += list(part)
assert set(union) == set(G.nodes())

pickle.dump(partition, open(folder + "graph_partitions.pickle", "wb"))

done with iteration at level: 2
done with iteration at level: 1
done with iteration at level: 1


### Partition Graph with k-Medoids

In [5]:
# import the knn graph
G = nx.read_gpickle(folder + 'knn_graph.gpickle')

points = pickle.load(open(folder + 'points.pickle', 'rb'))
distances = pickle.load(open(folder + 'distances.pickle', 'rb'))

In [11]:
def assign_all_points(meds):
    all_assignments = [set([]) for _ in range(M)]
    for i, point in enumerate(points):
        min_distance = distances[i, meds[0]]
        assignment = 0
        for j in range(1, M):
            new_distance = distances[i, meds[j]]
            if new_distance < min_distance:
                assignment = j
                min_distance = new_distance
        all_assignments[assignment].add(i)
    return all_assignments
    
def total_distance(med, cluster):
    total = 0
    for i in cluster:
        total += distances[i, med]
    return total

In [12]:
# initialize the medoid indices and partition
medoids = np.random.choice(range(len(points)), size=M, replace=False)
partition = assign_all_points(medoids)

# keep trying to make it better
at_least_one_changed = True
while at_least_one_changed:
    at_least_one_changed = False
    
    for i in range(len(medoids)):
        for new_med in partition[i]:
            if total_distance(new_med, partition[i]) < total_distance(medoids[i], partition[i]):
                at_least_one_changed = True
                medoids[i] = new_med
    
    if at_least_one_changed:
        partition = assign_all_points(medoids)

hi
hi


In [13]:
pickle.dump(partition, open(folder + 'km_partitions.pickle', 'wb'))
pickle.dump(medoids, open(folder + 'medoids.pickle', 'wb'))